In [11]:
# 기본 import
import os, time, datetime as dt
import requests
import pandas as pd

# 토큰 유틸 (그대로 사용)
from libs.kis_auth import get_or_load_access_token

# 환경 설정
APPKEY = os.getenv("KIS_API_KEY")
APPSECRET = os.getenv("KIS_API_SECRET")
ACCESS_TOKEN = get_or_load_access_token(env="real")

BASE = "https://openapi.koreainvestment.com:9443"

def headers(tr_id: str) -> dict:
    return {
        "content-type": "application/json; charset=utf-8",
        "authorization": f"Bearer {ACCESS_TOKEN}",
        "appkey": APPKEY,
        "appsecret": APPSECRET,
        "tr_id": tr_id,
        "custtype": "P",  # 개인
    }

def kis_get(url: str, tr_id: str, params: dict, retry=3, sleep=0.35):
    for _ in range(retry):
        r = requests.get(url, headers=headers(tr_id), params=params, timeout=15)
        if r.ok:
            return r.json()
        time.sleep(sleep)
    r.raise_for_status()

def to_num(x):
    if x is None: 
        return None
    try:
        return float(str(x).replace(",", ""))
    except Exception:
        return None

🔄 refreshed real token and saved to .env


In [37]:
##### 시세1
# https://apiportal.koreainvestment.com/apiservice-apiservice?/uapi/domestic-stock/v1/quotations/inquire-price
TR_QUOTE = "FHKST01010100"

QUOTE_KEEP = {
    "stck_shrn_iscd": "symbol",
    "rprs_mrkt_kor_name": "market",
    "bstp_kor_isnm": "industry",
    "stck_prpr": "price_now",
    "acml_vol": "volume_now",
    "acml_tr_pbmn": "value_now",
    "stck_mxpr": "upper_limit",
    "stck_llam": "lower_limit",
    "hts_avls": "market_cap",
    "prdy_vrss": "change",
    "prdy_ctrt": "change_rate",
}

QUOTE_NUMERIC = ["price_now","volume_now","value_now","upper_limit","lower_limit","market_cap","change","change_rate"]

def fetch_quote(symbol: str, market_code="J") -> dict:
    url = f"{BASE}/uapi/domestic-stock/v1/quotations/inquire-price"
    params = {"fid_cond_mrkt_div_code": market_code, "fid_input_iscd": str(symbol).zfill(6)}
    js = kis_get(url, TR_QUOTE, params)
    out = js.get("output") or {}
    rec = {dst: out.get(src) for src, dst in QUOTE_KEEP.items()}
    # 캐스팅
    rec["symbol"] = str(rec.get("symbol", symbol)).zfill(6)
    for k in QUOTE_NUMERIC:
        if k in rec:
            rec[k] = to_num(rec[k])
    return rec

##### 일별시세 
# https://apiportal.koreainvestment.com/apiservice-apiservice?/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice
TR_DAILY = "FHKST03010100"

DAILY_KEEP_O2 = {
    "stck_bsop_date": "date",
    "stck_oprc": "open",
    "stck_hgpr": "high",
    "stck_lwpr": "low",
    "stck_clpr": "close",
    "acml_vol": "volume",
    "acml_tr_pbmn": "value",
    # 정합성/이벤트 플래그(보존만)
    "flng_cls_code": "flng_cls_code",
    "prtt_rate": "split_ratio",
    "mod_yn": "mod_yn",
}

def date_chunks(start: str, end: str, span=100):
    d0 = dt.datetime.strptime(start, "%Y%m%d").date()
    d1 = dt.datetime.strptime(end, "%Y%m%d").date()
    cur = d0
    while cur <= d1:
        nxt = min(cur + dt.timedelta(days=span-1), d1)
        yield cur.strftime("%Y%m%d"), nxt.strftime("%Y%m%d")
        cur = nxt + dt.timedelta(days=1)

def fetch_daily(symbol: str, start: str, end: str, adjusted=True, market_code="J") -> pd.DataFrame:
    url = f"{BASE}/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice"
    rows = []
    meta_first = None
    for s,e in date_chunks(start, end, 100):
        params = {
            "fid_cond_mrkt_div_code": market_code,
            "fid_input_iscd": str(symbol).zfill(6),
            "fid_input_date_1": s,
            "fid_input_date_2": e,
            "fid_period_div_code": "D",
            "fid_org_adj_prc": "0" if adjusted else "1",
        }
        js = kis_get(url, TR_DAILY, params)
        if meta_first is None:
            meta_first = js.get("output1") or {}
        part = js.get("output2") or []
        rows.extend(part)
    if not rows:
        return pd.DataFrame()
    df = pd.DataFrame(rows).rename(columns=DAILY_KEEP_O2)[list(DAILY_KEEP_O2.values())]
    # 타입
    for c in ["open","high","low","close","volume","value","split_ratio"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d").dt.date
    # 메타에서 name 붙이기(가능하면)
    # fetch_daily 내부, 메타 붙이기 부분만 교체/보강
    name = None
    if meta_first:
        name = meta_first.get("hts_kor_isnm")
    df.insert(0, "symbol", str(symbol).zfill(6))
    df.insert(1, "name", name if name else None)   # 항상 name 컬럼 생성
    df["adj_flag"] = True if adjusted else False
    df = df.sort_values("date").drop_duplicates(subset=["date"]).reset_index(drop=True)
    return df


def build_raw_frame(symbols, start="20150101", end=None):
    if end is None:
        end = dt.date.today().strftime("%Y%m%d")

    # 1) 시세1 스냅샷
    quote_rows = []
    for s in symbols:
        try:
            quote_rows.append(fetch_quote(s))
        except Exception as e:
            quote_rows.append({"symbol": str(s).zfill(6), "error": str(e)})
    qdf = pd.DataFrame(quote_rows).drop_duplicates(subset=["symbol"])

    # 2) 일별시세
    frames = []
    for s in symbols:
        try:
            frames.append(fetch_daily(s, start, end, adjusted=True))
        except Exception as e:
            frames.append(pd.DataFrame([{"symbol": str(s).zfill(6), "error": str(e)}]))
    ddf = pd.concat(frames, ignore_index=True)

    # 2-1) ddf에 name 없을 경우 대비해 빈 컬럼 생성
    if "name" not in ddf.columns:
        ddf["name"] = None

    # 3) 조인 (시세1 메타 브로드캐스트) — 여기서 qdf에 name 안 붙임
    meta_cols = ["symbol","market","industry","market_cap","change","change_rate",
                 "price_now","volume_now","value_now"]
    meta_cols = [c for c in meta_cols if c in qdf.columns]
    out = ddf.merge(qdf[meta_cols], on="symbol", how="left")

    # 4) 전일 종가
    if {"symbol","date","close"}.issubset(out.columns):
        out["prdy_close"] = out.groupby("symbol")["close"].shift(1)
    else:
        out["prdy_close"] = None

    # 5) 오늘자 덮어쓰기
    today = dt.date.today()
    mask_today = (out["date"] == today) if "date" in out.columns else False
    for src, dst in [("price_now","close"), ("volume_now","volume"), ("value_now","value")]:
        if src in out.columns and dst in out.columns:
            out.loc[mask_today & out[src].notna(), dst] = out.loc[mask_today, src]

    # 6) 최종 컬럼 정리 (없으면 생성)
    cols = ["symbol","date","open","high","low","close","volume","value","adj_flag",
            "name","market","industry","market_cap","prdy_close","change","change_rate"]
    for c in cols:
        if c not in out.columns:
            out[c] = None

    # 타입 캐스팅
    for c in ["open","high","low","close","volume","value","market_cap","prdy_close","change","change_rate"]:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors="coerce")

    # 정렬
    out = out[cols].sort_values(["symbol","date"]).reset_index(drop=True)
    return out

######## 실행 (업데이트) ########
# 심볼 로드
symbols = pd.read_csv("data/meta/top50_symbols.txt", header=None)[0].astype(str).str.zfill(6).tolist()

# 기간
start = dt.date.today() - dt.timedelta(days=365*5)  # 5년치
start = start.strftime("%Y%m%d")
end   = dt.date.today().strftime("%Y%m%d")

# 1) 일봉+시세1 취합 (기존)
raw_df = build_raw_frame(symbols, start=start, end=end)

display(raw_df.head(10))

,symbol,date,open,high,low,close,volume,value,adj_flag,name,market,industry,market_cap,prdy_close,change,change_rate
0,000020,2020-09-28,22100,22500,21500,22000,442765,9716150700,True,동화약품,KOSPI,제약,1779.0,NaN,-40.0,-0.62
1,000020,2020-09-29,22000,22050,20700,21850,551123,11830525650,True,동화약품,KOSPI,제약,1779.0,22000.0,-40.0,-0.62
2,000020,2020-10-05,22200,24350,21700,24100,1094057,25443501900,True,동화약품,KOSPI,제약,1779.0,21850.0,-40.0,-0.62
3,000020,2020-10-06,24100,24350,23350,23750,463912,10997525600,True,동화약품,KOSPI,제약,1779.0,24100.0,-40.0,-0.62
4,000020,2020-10-07,23500,23550,23100,23500,322425,7531389650,True,동화약품,KOSPI,제약,1779.0,23750.0,-40.0,-0.62
5,000020,2020-10-08,23600,24850,23600,24150,716608,17398574600,True,동화약품,KOSPI,제약,1779.0,23500.0,-40.0,-0.62
6,000020,2020-10-12,24500,24950,22950,24000,540131,13038907700,True,동화약품,KOSPI,제약,1779.0,24150.0,-40.0,-0.62
7,000020,2020-10-13,24200,24550,23900,24300,426678,10341755400,True,동화약품,KOSPI,제약,1779.0,24000.0,-40.0,-0.62
8,000020,2020-10-14,23900,24200,23600,23850,420287,10025795900,True,동화약품,KOSPI,제약,1779.0,24300.0,-40.0,-0.62
9,000020,2020-10-15,23850,23850,22450,22750,508031,11652163150,True,동화약품,KOSPI,제약,1779.0,23850.0,-40.0,-0.62


In [55]:
# =========================
# 투자자 일별: 수집 → 내부비율 계산 → 조인 → 클린 저장
# =========================
import os, time, datetime as dt
import requests
import pandas as pd

# ---- KIS headers / GET with headers ----
TR_STOCK_INV_DAILY = "FHPTJ04160001"

def headers(tr_id: str, env: str = "real"):
    return {
        "content-type": "application/json; charset=utf-8",
        "authorization": f"Bearer {ACCESS_TOKEN}",   # 전역 준비(혹은 get_or_load_access_token 사용)
        "appkey": APPKEY,
        "appsecret": APPSECRET,
        "tr_id": tr_id,
        "custtype": "P",  # 개인
    }

def kis_get_with_headers(url: str, tr_id: str, params: dict, retry=3, sleep=0.35):
    last_err = None
    for _ in range(retry):
        try:
            r = requests.get(url, headers=headers(tr_id), params=params, timeout=15)
            if r.ok:
                return r.json(), r.headers
            last_err = RuntimeError(f"HTTP {r.status_code}: {r.text[:200]}")
        except Exception as e:
            last_err = e
        time.sleep(sleep)
    raise last_err if last_err else RuntimeError("Unknown KIS request error")

# ---- 스키마 매핑(필요 필드만) ----
INV_DAILY_KEEP = {
    "stck_bsop_date": "date",
    "acml_vol": "total_volume",          # (전체) 거래량(주) - 참고용
    "acml_tr_pbmn": "total_value_mn",    # (전체) 거래대금(백만원) - 참고용

    # 개인
    "prsn_shnu_vol": "prsn_buy_vol",
    "prsn_seln_vol": "prsn_sell_vol",
    "prsn_shnu_tr_pbmn": "prsn_buy_val_mn",
    "prsn_seln_tr_pbmn": "prsn_sell_val_mn",

    # 외국인
    "frgn_shnu_vol": "frgn_buy_vol",
    "frgn_seln_vol": "frgn_sell_vol",
    "frgn_shnu_tr_pbmn": "frgn_buy_val_mn",
    "frgn_seln_tr_pbmn": "frgn_sell_val_mn",

    # 기관
    "orgn_shnu_vol": "orgn_buy_vol",
    "orgn_seln_vol": "orgn_sell_vol",
    "orgn_shnu_tr_pbmn": "orgn_buy_val_mn",
    "orgn_seln_tr_pbmn": "orgn_sell_val_mn",
}

INV_DAILY_NUM = [
    "total_volume","total_value_mn",
    "prsn_buy_vol","prsn_sell_vol","prsn_buy_val_mn","prsn_sell_val_mn",
    "frgn_buy_vol","frgn_sell_vol","frgn_buy_val_mn","frgn_sell_val_mn",
    "orgn_buy_vol","orgn_sell_vol","orgn_buy_val_mn","orgn_sell_val_mn",
]

# ---- 내부비율(개인/외인/기관 집단 내부 합=1) 계산 ----
def _sd(a, b):
    try:
        return float(a) / float(b) if (a is not None and b not in (None, 0)) else None
    except Exception:
        return None

def compute_group_ratios_strict(df: pd.DataFrame) -> pd.DataFrame:
    """
    3집단(개인/외국인/기관) 내부 비율:
      - 대금: buy/sell/net
        * buy_ratio  = grp_buy_val / (prsn+frgn+orgn buy_val 합)
        * sell_ratio = grp_sell_val / (prsn+frgn+orgn sell_val 합)
        * net_ratio  = (grp_buy_val - grp_sell_val) / (buy_val_sum + sell_val_sum)
      - 거래량: buy/sell/net
        * buy_ratio  = grp_buy_vol / (prsn+frgn+orgn buy_vol 합)
        * sell_ratio = grp_sell_vol / (prsn+frgn+orgn sell_vol 합)
        * net_ratio  = (grp_buy_vol - grp_sell_vol) / (buy_vol_sum + sell_vol_sum)

    → 각 날짜별로 buy/sell 비율은 3집단 합이 1.0, net은 [-1, 1]로 안정.
    """
    out = df.copy()

    # 숫자 변환
    num_cols = [
        "prsn_buy_val_mn","prsn_sell_val_mn","frgn_buy_val_mn","frgn_sell_val_mn","orgn_buy_val_mn","orgn_sell_val_mn",
        "prsn_buy_vol","prsn_sell_vol","frgn_buy_vol","frgn_sell_vol","orgn_buy_vol","orgn_sell_vol",
    ]
    for c in num_cols:
        if c in out.columns:
            out[c] = pd.to_numeric(out[c], errors="coerce")

    # 내부 분모
    out["buy_val_sum"]  = out[["prsn_buy_val_mn","frgn_buy_val_mn","orgn_buy_val_mn"]].sum(axis=1, min_count=1)
    out["sell_val_sum"] = out[["prsn_sell_val_mn","frgn_sell_val_mn","orgn_sell_val_mn"]].sum(axis=1, min_count=1)
    out["val_den"]      = out["buy_val_sum"] + out["sell_val_sum"]

    out["buy_vol_sum"]  = out[["prsn_buy_vol","frgn_buy_vol","orgn_buy_vol"]].sum(axis=1, min_count=1)
    out["sell_vol_sum"] = out[["prsn_sell_vol","frgn_sell_vol","orgn_sell_vol"]].sum(axis=1, min_count=1)
    out["vol_den"]      = out["buy_vol_sum"] + out["sell_vol_sum"]

    keep = ["symbol","date"]
    for grp in ["prsn","frgn","orgn"]:
        # 금액 비율
        out[f"{grp}_buy_val_ratio"]  = out.apply(lambda r: _sd(r[f"{grp}_buy_val_mn"],  r["buy_val_sum"]),  axis=1)
        out[f"{grp}_sell_val_ratio"] = out.apply(lambda r: _sd(r[f"{grp}_sell_val_mn"], r["sell_val_sum"]), axis=1)
        out[f"{grp}_net_val_ratio"]  = out.apply(lambda r: _sd((r[f"{grp}_buy_val_mn"]-r[f"{grp}_sell_val_mn"]), r["val_den"]), axis=1)
        # 거래량 비율
        out[f"{grp}_buy_vol_ratio"]  = out.apply(lambda r: _sd(r[f"{grp}_buy_vol"],    r["buy_vol_sum"]),  axis=1)
        out[f"{grp}_sell_vol_ratio"] = out.apply(lambda r: _sd(r[f"{grp}_sell_vol"],   r["sell_vol_sum"]), axis=1)
        out[f"{grp}_net_vol_ratio"]  = out.apply(lambda r: _sd((r[f"{grp}_buy_vol"]-r[f"{grp}_sell_vol"]),   r["vol_den"]), axis=1)

        keep += [
            f"{grp}_buy_val_ratio", f"{grp}_sell_val_ratio", f"{grp}_net_val_ratio",
            f"{grp}_buy_vol_ratio", f"{grp}_sell_vol_ratio", f"{grp}_net_vol_ratio",
        ]

    res = out[keep].sort_values(["symbol","date"]).reset_index(drop=True)
    return res

# ---- 종목별 투자자매매동향(일별) 수집: 앵커-루프 ----
def fetch_investor_trade_by_stock_daily(symbol: str,
                                        start: str,
                                        end: str,
                                        market_code: str = "J",
                                        max_loops: int = 200,
                                        pause: float = 0.25) -> pd.DataFrame:
    """
    FID_INPUT_DATE_1를 기준(anchor)으로 header.tr_cont 페이징.
    한 anchor 수집 후, 그 배치의 최소 영업일(min_d) 이전으로 anchor를 옮겨 반복.
    start까지 내려가면 중단.
    """
    url = f"{BASE}/uapi/domestic-stock/v1/quotations/investor-trade-by-stock-daily"

    def _one_anchor(anchor_yyyymmdd: str):
        params = {
            "fid_cond_mrkt_div_code": market_code,
            "fid_input_iscd": str(symbol).zfill(6),
            "fid_input_date_1": anchor_yyyymmdd,
            "fid_org_adj_prc": "",
            "fid_etc_cls_code": "",
        }
        batch_rows = []
        while True:
            js, hdr = kis_get_with_headers(url, TR_STOCK_INV_DAILY, params)
            rows = js.get("output2") or []
            batch_rows.extend(rows)

            trc = (hdr.get("tr_cont") or hdr.get("Tr_Cont") or "").strip()
            if trc == "M":
                params["tr_cont"] = "N"
            else:
                break
            if len(batch_rows) > 20000:
                break
            time.sleep(pause)
        return batch_rows

    all_rows = []
    anchor = end
    start_d = pd.to_datetime(start, format="%Y%m%d").date()
    loops = 0

    while loops < max_loops:
        loops += 1
        rows = _one_anchor(anchor)
        if not rows:
            break
        all_rows.extend(rows)

        tmp = pd.DataFrame(rows)
        if "stck_bsop_date" not in tmp.columns or tmp.empty:
            break
        tmp["date"] = pd.to_datetime(tmp["stck_bsop_date"], format="%Y%m%d").dt.date
        min_d = tmp["date"].min()

        if min_d <= start_d:
            break

        anchor = (min_d - pd.Timedelta(days=1)).strftime("%Y%m%d")
        time.sleep(pause)

    if not all_rows:
        return pd.DataFrame(columns=["symbol","date"])

    df = pd.DataFrame(all_rows).rename(columns=INV_DAILY_KEEP)[list(INV_DAILY_KEEP.values())]
    df.insert(0, "symbol", str(symbol).zfill(6))
    for c in INV_DAILY_NUM:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    df["date"] = pd.to_datetime(df["date"], format="%Y%m%d").dt.date
    df = df.sort_values(["symbol","date"]).drop_duplicates(["symbol","date"]).reset_index(drop=True)
    return df

# =========================
# 실행: 수집 → 비율 → 조인 → 클린 → 저장
# =========================

# 0) 대상 심볼/기간(raw_df 기준)
symbols = sorted(raw_df["symbol"].astype(str).str.zfill(6).unique().tolist())
dmin = pd.to_datetime(raw_df["date"]).min().strftime("%Y%m%d")
dmax = pd.to_datetime(raw_df["date"]).max().strftime("%Y%m%d")

# 1) 수집 & 내부비율 계산
ratio_frames = []
for s in symbols:
    try:
        raw_one = fetch_investor_trade_by_stock_daily(s, start=dmin, end=dmax, market_code="J")
        if raw_one.empty:
            continue
        ratio_one = compute_group_ratios_strict(raw_one)
        ratio_frames.append(ratio_one)
    except Exception as e:
        print(f"[WARN] {s}: {e}")

inv_ratio_df = pd.concat(ratio_frames, ignore_index=True) if ratio_frames else pd.DataFrame(columns=["symbol","date"])

# 2) 조인(좌조인: raw_df 기준)
merged_df = (
    raw_df
    .copy()
    .assign(symbol=lambda d: d["symbol"].astype(str).str.zfill(6),
            date=lambda d: pd.to_datetime(d["date"]).dt.date)
    .merge(inv_ratio_df, on=["symbol","date"], how="left")
    .sort_values(["symbol","date"])
    .reset_index(drop=True)
)

# 3) 클린업
# 3-1) 투자자 비율 컬럼 목록
RATIO_COLS = []
for g in ["prsn","frgn","orgn"]:
    RATIO_COLS += [
        f"{g}_buy_val_ratio", f"{g}_sell_val_ratio", f"{g}_net_val_ratio",
        f"{g}_buy_vol_ratio", f"{g}_sell_vol_ratio", f"{g}_net_vol_ratio",
    ]

# 3-2) 투자자비율 전무한 종목 제거
sym_has_any = merged_df.groupby("symbol")[RATIO_COLS].apply(lambda x: x.notna().any().any())
keep_syms = sym_has_any[sym_has_any].index.tolist()
clean_df = merged_df[merged_df["symbol"].isin(keep_syms)].copy()

# 3-3) 날짜 커버리지 100% 유지(남은 종목 모두에서 비율값이 있는 날짜만)
clean_df["_complete_row"] = clean_df[RATIO_COLS].notna().all(axis=1)

n_syms_now = clean_df["symbol"].nunique()
date_ok_cnt = clean_df.groupby("date")["_complete_row"].sum().rename("ok_cnt")
keep_dates = date_ok_cnt[date_ok_cnt == n_syms_now].index.tolist()

clean_df = (
    clean_df[clean_df["date"].isin(keep_dates)]
    .drop(columns=["_complete_row"])
    .sort_values(["date","symbol"])
    .reset_index(drop=True)
)

# 4) 저장
os.makedirs("data/proc", exist_ok=True)
out_parquet = "data/proc/daily_raw_clean.parquet"
out_csv     = "data/proc/daily_raw_clean.csv"

clean_df.to_parquet(out_parquet, index=False)
clean_df.to_csv(out_csv, index=False, encoding="utf-8-sig")

print(f"[DONE] 남은 심볼={clean_df['symbol'].nunique()} | 남은 날짜={clean_df['date'].nunique()} | 행수={len(clean_df):,}")
display(clean_df.head(3))
display(clean_df.tail(3))


[WARN] 277810: '<=' not supported between instances of 'datetime.date' and 'float'
[WARN] 278470: '<=' not supported between instances of 'datetime.date' and 'float'
[WARN] 440110: '<=' not supported between instances of 'datetime.date' and 'float'
[DONE] 남은 심볼=47 | 남은 날짜=1141 | 행수=53,627


,symbol,date,open,high,low,close,volume,value,adj_flag,name,...,frgn_net_val_ratio,frgn_buy_vol_ratio,frgn_sell_vol_ratio,frgn_net_vol_ratio,orgn_buy_val_ratio,orgn_sell_val_ratio,orgn_net_val_ratio,orgn_buy_vol_ratio,orgn_sell_vol_ratio,orgn_net_vol_ratio
0,000020,2020-09-28,22100,22500,21500,22000,442765,9716150700,True,동화약품,...,-0.051862,0.088291,0.181377,-0.051509,0.027809,0.029539,-0.001921,0.027868,0.029513,-0.001879
1,000070,2020-09-28,62200,63200,62200,62500,4212,263495000,True,삼양홀딩스,...,-0.079848,0.127255,0.285138,-0.078941,0.406844,0.353612,0.026616,0.404796,0.354701,0.025047
2,000080,2020-09-28,33450,36100,33450,35950,493071,17551153600,True,하이트진로,...,0.119646,0.384996,0.142985,0.120592,0.272306,0.101709,0.085005,0.271440,0.101331,0.084762


,symbol,date,open,high,low,close,volume,value,adj_flag,name,...,frgn_net_val_ratio,frgn_buy_vol_ratio,frgn_sell_vol_ratio,frgn_net_vol_ratio,orgn_buy_val_ratio,orgn_sell_val_ratio,orgn_net_val_ratio,orgn_buy_vol_ratio,orgn_sell_vol_ratio,orgn_net_vol_ratio
53624,298380,2025-09-26,93100,93800,91400,91500,81956,7565024150,True,에이비엘바이오,...,0.019164,0.272369,0.231285,0.019586,0.042329,0.268818,-0.113837,0.042547,0.270442,-0.114542
53625,353200,2025-09-26,28400,28600,28100,28250,141953,4027604200,True,대덕전자,...,-0.073493,0.176801,0.328284,-0.073410,0.152269,0.035845,0.059073,0.151615,0.036061,0.058643
53626,357780,2025-09-26,285000,289500,282000,289000,5748,1644084500,True,솔브레인,...,-0.000171,0.333339,0.328459,0.001825,0.436308,0.133333,0.150969,0.433123,0.135070,0.148499


In [56]:
clean_df.iloc[-1, :]

symbol                     357780
date                   2025-09-26
open                       285000
high                       289500
low                        282000
close                      289000
volume                       5748
value                  1644084500
adj_flag                     True
name                         솔브레인
market                     KSQ150
industry                       화학
market_cap                22480.0
prdy_close               293000.0
change                    -4000.0
change_rate                 -1.37
prsn_buy_val_ratio        0.23313
prsn_sell_val_ratio      0.536965
prsn_net_val_ratio      -0.152618
prsn_buy_vol_ratio       0.233538
prsn_sell_vol_ratio       0.53647
prsn_net_vol_ratio      -0.152182
frgn_buy_val_ratio       0.330562
frgn_sell_val_ratio      0.329702
frgn_net_val_ratio      -0.000171
frgn_buy_vol_ratio       0.333339
frgn_sell_vol_ratio      0.328459
frgn_net_vol_ratio       0.001825
orgn_buy_val_ratio       0.436308
orgn_sell_val_